# 第一种使用GPU训练的方法

- 网络模型.cuda()
- 损失函数.cuda()
- 数据（输入，标注）.cuda

In [1]:
import torchvision
from torch.utils.data import DataLoader
from torch import nn
import torch
import time

# 1. 准备训练数据集
train_data = torchvision.datasets.CIFAR10(
    root='./datasets',
      train=True, 
      transform=torchvision.transforms.ToTensor(), 
      download=True
)


# 2. 准备测试数据集
test_data = torchvision.datasets.CIFAR10(
    root='./datasets',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

train_data_size = len(train_data) # 训练数据集的大小 50000张
test_data_size = len(test_data)   # 测试数据集的大小 10000张


# 3. 利用Dataloader 加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


# 4. 搭建神经网络
# 因为 CIRAR10 是一个 10 分类的数据集，所以输出的类别是 10 个
# 这个可以单独放到一个文件中，比如model.py，然后导入这个包
class TuDui(nn.Module):
    def __init__(self):
        super(TuDui, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        output = self.model(x)
        return output


# 5. 创建网络模型
tudui = TuDui()
if torch.cuda.is_available():
    tudui = tudui.cuda()  # 这里开始使用GPU训练（模型可以调用）
    

# 6. 创建损失函数
loss_function = nn.CrossEntropyLoss()  # CrossEntropyLoss这个损失常用于分类问题
loss_function = loss_function.cuda()   # 使用GPU训练（损失函数也可以调用）


# 7. 创建优化器
learning_rate = 0.001 # 学习率， 这个通常单独作为一个变量传入优化器,  便于修改
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate, )

In [4]:
# 8. 设置训练网络的一些参数
total_train_step = 0  # 记录训练的总次数
total_test_step = 0   # 记录测试的总次数
epoch = 10            # 训练的轮数

start_time = time.time()  # 记录开始训练的时间
for i in range(epoch):
    print("-----------------第 {} 轮训练开始-----------------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        img, target = data
        img = img.cuda()  # 调用GPU
        target = target.cuda() # 调用GPU
        output = tudui(img) # 投喂到神经网络中得到一个输出
        loss = loss_function(output, target)  # 计算损失（预测输出和真实值之间的关系）

        # 开始优化（利用优化器优化模型）
        optimizer.zero_grad()  # 优化之前一定要先把梯度清零
        loss.backward()   # 利用损失函数的反向传播，得到每个参数节点的梯度
        optimizer.step()  # 对其中的参数进行一个优化

        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            end_time = time.time()
            print('time: {:.4f}'.format(end_time-start_time))
            print("训练次数：{}，Loss：{:.4f}".format(total_train_step, loss.item()))
        

    # 上边是完整的一轮训练流程
    # 当一轮训练结束后，如何来判断训练的结果到底怎么样，有没有达到预期，需要对训练的结果进行测试
    # 根据测试数据集上的正确率/损失，来判断模型有没有训练好
    # 测试的过程就不要调优了，需要根据现有模型来测试
    total_test_loss = 0  # 本轮测试的总损失
    with torch.no_grad():
        for data in test_dataloader:
            img, target = data
            img = img.cuda()
            target = target.cuda()
            output = tudui(img) # 将测试数据投喂到网络中
            loss = loss_function(output,target)  # 计算模型输出与真实值之间的误差
            total_test_loss = total_test_loss + loss.item()
    print("整体测试集上的Loss:{:.4f}.".format(total_test_loss))

-----------------第 1 轮训练开始-----------------
time: 0.0124
time: 0.0303
time: 0.0451
time: 0.0583
time: 0.0707
time: 0.0821
time: 0.0927
time: 0.1026
time: 0.1122
time: 0.1212
time: 0.1299
time: 0.1385
time: 0.1472
time: 0.1559
time: 0.1646
time: 0.1733
time: 0.1820
time: 0.1909
time: 0.1998
time: 0.2095
time: 0.2185
time: 0.2272
time: 0.2359
time: 0.2446
time: 0.2533
time: 0.2620
time: 0.2707
time: 0.2794
time: 0.2881
time: 0.2968
time: 0.3054
time: 0.3141
time: 0.3228
time: 0.3315
time: 0.3402
time: 0.3489
time: 0.3576
time: 0.3663
time: 0.3750
time: 0.3837
time: 0.3924
time: 0.4011
time: 0.4098
time: 0.4192
time: 0.4280
time: 0.4367
time: 0.4455
time: 0.4542
time: 0.4629
time: 0.4715
time: 0.4802
time: 0.4889
time: 0.4977
time: 0.5063
time: 0.5150
time: 0.5238
time: 0.5325
time: 0.5412
time: 0.5500
time: 0.5587
time: 0.5674
time: 0.5761
time: 0.5848
time: 0.5935
time: 0.6021
time: 0.6108
time: 0.6195
time: 0.6289
time: 0.6378
time: 0.6465
time: 0.6552
time: 0.6639
time: 0.6727
time: 0